  Created wheel for playsound: filename=playsound-1.3.0-py3-none-any.whl size=7025 sha256=7e2435a59d728c29611da9f396858159f18af8e1887ff84c571243bd61021011
  Stored in directory: /home/gauffrea/.cache/pip/wheels/73/cd/cf/9750b618d54bd81c20e4c34fb24a423a5b095920367cdb3f71
Successfully built playsound


In [77]:
# !pip3 install gspread
# !pip3 install --upgrade google-api-python-client oauth2client 
# !pip3 install playsound
# https://docs.google.com/spreadsheets/d/1RlPPoT6HQkQbo_607QJRJPetRsBjRhnD97N01EgK9x0/edit#gid=0

In [90]:
import requests 
import json
import gspread
import os
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials
from playsound import playsound

import time

In [79]:
def date_to_url(s):
    """ input 17/05/1996 
        return 19962F052F17 (url format)"""
    l = s.split('/')
    if len(l)==2:
        year=2022
        day,month = l
    elif len(l)==3:
        day,month,year = l

    return f'{year}%2F{month}%2F{day}'
def time_to_int(date):
    return int(date.split(':')[0])
def get_number_of_row(sh):
    return len(sh.col_values(1))

In [92]:
time.sleep(3)
print('bite')
time.sleep(3)
print('bite')
time.sleep(3)
print('bite')

bite
bite
bite


In [ ]:
while True: 
    
    gc = gspread.service_account(filename=os.path.abspath('istheretgvmax-5273da7c9b7a.json')) #Put the path of your own json google token
    sh = gc.open("is-there-tgvmax")
    worksheet_list = [sh for sh in sh.worksheets() if '@' in sh.title]

    sh=worksheet_list[0]
    row_values = range(2,get_number_of_row(sh)+1)
    for row_value in row_values:
        origine,destination,date,heure_min,heure_max,_,email_envoye,force_update=sh.row_values(row_value)
        print(origine,destination,date,heure_min,heure_max,_,email_envoye,force_update)
        request_start = 'https://ressources.data.sncf.com/api/records/1.0/search/?dataset=tgvmax&q=&sort=-date&facet=date&facet=origine&facet=destination&facet=od_happy_card'
        full_request = f'{request_start}&refine.destination={destination}&refine.origine={origine}&refine.date={date_to_url(date)}'
        request=requests.get(full_request)
        dico = json.loads(request.text)
        voyages = dico['records']


        nb_trouve = 0
        print(full_request)
        for (i,voyage) in enumerate(voyages):
            heure_depart = time_to_int(voyage['fields']['heure_depart']) 
            if (heure_depart>=int(heure_min) and heure_depart<=int(heure_max))and voyage['fields']['od_happy_card']=='OUI' :

                nb_trouve +=1
        print(len(voyages),nb_trouve)
        sh.update(f'F{row_value}', nb_trouve)
    if nb_trouve>0:
        playsound("/home/gauffrea/Documents/song.mp3")
    time.sleep(120)

PARIS+(intramuros) BORDEAUX+ST+JEAN 28/06/2022 0 24 3 FALSE FALSE
https://ressources.data.sncf.com/api/records/1.0/search/?dataset=tgvmax&q=&sort=-date&facet=date&facet=origine&facet=destination&facet=od_happy_card&refine.destination=BORDEAUX+ST+JEAN&refine.origine=PARIS+(intramuros)&refine.date=2022%2F06%2F28
10 0
PARIS+(intramuros) BORDEAUX+ST+JEAN 28/06/2022 0 24 0 FALSE FALSE
https://ressources.data.sncf.com/api/records/1.0/search/?dataset=tgvmax&q=&sort=-date&facet=date&facet=origine&facet=destination&facet=od_happy_card&refine.destination=BORDEAUX+ST+JEAN&refine.origine=PARIS+(intramuros)&refine.date=2022%2F06%2F28
10 0
PARIS+(intramuros) BORDEAUX+ST+JEAN 28/06/2022 0 24 0 FALSE FALSE
https://ressources.data.sncf.com/api/records/1.0/search/?dataset=tgvmax&q=&sort=-date&facet=date&facet=origine&facet=destination&facet=od_happy_card&refine.destination=BORDEAUX+ST+JEAN&refine.origine=PARIS+(intramuros)&refine.date=2022%2F06%2F28
10 0
PARIS+(intramuros) BORDEAUX+ST+JEAN 28/06/2022 0 

In [81]:
voyages

[{'datasetid': 'tgvmax',
  'recordid': '68ccbae5c45f57e19ff2e8d5f7e33c8a71677e92',
  'fields': {'heure_arrivee': '19:42',
   'code_equip': 'TGF',
   'date': '2022-06-28',
   'train_no': '6921',
   'destination': 'GRENOBLE',
   'origine': 'PARIS (intramuros)',
   'entity': 'PAGRENOBLE',
   'origine_iata': 'FRPLY',
   'od_happy_card': 'OUI',
   'axe': 'SUD-EST',
   'destination_iata': 'FRGNB',
   'heure_depart': '16:42'},
  'record_timestamp': '2022-06-27T10:00:31.979Z'},
 {'datasetid': 'tgvmax',
  'recordid': '8b897c73aa9877dc801a22094726c12457f5e5fe',
  'fields': {'heure_arrivee': '17:46',
   'code_equip': 'TGF',
   'date': '2022-06-28',
   'train_no': '6919',
   'destination': 'GRENOBLE',
   'origine': 'PARIS (intramuros)',
   'entity': 'PAGRENOBLE',
   'origine_iata': 'FRPLY',
   'od_happy_card': 'NON',
   'axe': 'SUD-EST',
   'destination_iata': 'FRGNB',
   'heure_depart': '14:43'},
  'record_timestamp': '2022-06-27T10:00:31.979Z'},
 {'datasetid': 'tgvmax',
  'recordid': '7d6aeb4c9a

KeyboardInterrupt: 